# Energy calibration using the AgI+CuI 'standard'

Perfrom a timescan, e.g. timescan 1, 1, on the AgI+CuI 'standard' and update fname to the scan
Set the used photon energy

This notebook will 
1) Search for the possible peaks (i.e. the ones allowed under the used photon energy), 
2) Fit each one with a Gaussian to find the center
3) Perform a second order fit of the found chanel numbers vs. known edge energies
4) Return the found calibration values for pyMCA

**This work best with 35 keV - much less accurate results if I peaks are not included!**

In [ ]:
fname = '/data/visitors/danmax/PROPOSAL/VISIT/raw/SAMPLE/scan-XXXX.h5'
ph_energy = 35 # Photon energy in keV

################################################################################

%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as sci_op
import h5py

# Peak profile - in this case gaussian
def gauss(x,a,x0,sigma,b):
    '''
    Simple Gaussian function
    '''
    return a*np.exp(-(x-x0)**2/(2*sigma**2))+b

def e2ch_default(energy):
    '''
    Default/rough conversion from energy to detector chanels
    '''
    return energy/0.0124

# Open file and extract XRF data
fh = h5py.File(fname, 'r')
falconx = np.sum(fh['/entry/instrument/falconx/data'], axis=0)

# Set up all edges (names and energies) and pick out the ones that is under the photon energy used (and specified above
_labels = np.array(['CuKa', 'CuKb', 'AgKa', 'AgKb', 'IKa','IKb']) # All edge names
_energies = np.array([8.046, 8.904, 22.163, 24.941, 28.612, 32.294]) # All edge energies in keV
used = np.where(_energies <= ph_energy)[0] # Indices where edge should appear in data
energies = _energies[used] # Possible edge energies
labels = _labels[used] # Possible edge names

fit_ch = [] # Empty list to store the fitted edge chanels

ch_delta = 50 # Half-width of window where data is fitted
for e, name in zip(energies, labels):
    calc_ch = int(e2ch_default(e)) # Calculated chanel for edge - need to be used for offset
    peak_data = falconx[calc_ch-ch_delta: calc_ch+ch_delta] # Extract data to fit
    peak_ch = np.linspace(0, len(peak_data)-1, len(peak_data)) # Create a new chanel vector

    # Gueses for starting parameters
    guess_center = np.argmax(peak_data) # Find center for initial parameter guess
    guess_b = (peak_data[0]+peak_data[-1])/2 # Guess for background
    guess_sigma = 5 # Guess for sigma
    
    convergance = True # Assume convergance before fit

    # Fit the peak
    try:
        popt,pcov = sci_op.curve_fit(gauss, peak_ch, peak_data,p0=[np.max(peak_data)-guess_b,guess_center,guess_sigma,guess_b])
        fit_ch.append(popt[1]+calc_ch-ch_delta) # Store the fitted channel value for center of the peak
    except RuntimeError:
        print(f'Fit did not converge for {name}!')

# Plot the data and the fitted positions
fig, ax = plt.subplots()
ax.plot(falconx)
ax.scatter(fit_ch, falconx[np.array(fit_ch, dtype=int)], s=50, marker='o', ec='red', fc='None')
for i, label in enumerate(labels):
    ax.annotate(label, (fit_ch[i], falconx[np.array(fit_ch, dtype=int)][i]))
ax.semilogy()

# Plot the fitted datapoints
fig, ax = plt.subplots()
ax.scatter(fit_ch, energies)
for i, label in enumerate(labels):
    ax.annotate(label, (fit_ch[i], energies[i]+1))
ax.set_ylim(np.min(energies)-1, np.max(energies)+2)

#Second order fit
fit = np.polyfit(fit_ch, energies, 2)
fit_chanels = np.linspace(fit_ch[0], fit_ch[-1], 100)
ax.plot(fit_chanels, fit[0]*fit_chanels**2+fit[1]*fit_chanels+fit[2], 'r-') # Plot this

plt.ylabel('Energy / keV')
plt.xlabel('Chanel #')
plt.tight_layout()

print(f'In PyMCA use: \nA: {fit[2]:.6f} B: {fit[1]:.6f} C: {fit[0]:.6f}') 

# Manual calibration

Below you can fill in edge names, fitted peak positions, and energies manually and perform a fit

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt

labels = ['NiKa', 'NiKb', 'CuKa', 'CuKb','ZrKa','ZrKb','AgKa', 'AgKb']
chanels = [611.506, 675.794, 657.846, 727.921, 1283.26, 1438.52, 1799.41, 2026.77]
energy = [7.48, 8.267, 8.046, 8.904, 15.775, 17.668, 22.163, 24.941]

#Change the offset for the chanels
chanels = [chanel-1 for chanel in chanels] 

#Second order fit
fig, ax = plt.subplots()
ax.scatter(chanels, energy)
for i, label in enumerate(labels):
    ax.annotate(label, (chanels[i], energy[i]+1))
ax.set_ylim(np.min(energy)-1, np.max(energy)+2)
fit = np.polyfit(chanels, energy, 2)
fit_chanels = np.linspace(chanels[0], chanels[-1], 100)
ax.plot(fit_chanels, fit[0]*fit_chanels**2+fit[1]*fit_chanels+fit[2], 'r-')
print(f'In PyMCA use: \nA: {fit[2]:.6f} B: {fit[1]:.6f} C: {fit[0]:.6f}') 